In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import Draw
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
from tqdm import tqdm
from joblib import Parallel, delayed

In [ ]:
# read original dataset
data_fia49k = pd.read_csv(r"..\data\FIA53k.csv", low_memory=False)

In [ ]:
# take only every 4th entry to save time
data = data_fia49k["la_smiles"].values[::4]

In [ ]:
# generate mol objects from smiles
mols = []
formulars = []

for smiles in tqdm(data):
    mol = Chem.MolFromSmiles(smiles)
    mols.append(mol)
    formulars.append(CalcMolFormula(mol))

In [ ]:
Draw.MolToImage(mols[0])

In [ ]:
# calculate the mol properties in paralell
descrs = Parallel(n_jobs=-1)(
    delayed(Descriptors.CalcMolDescriptors)(mol) for mol in tqdm(mols)
)

# filter interesting parameters
interesting_descriptors = [
    "MolWt",
    "ExactMolWt",
    "NumValenceElectrons",
    "HeavyAtomCount",
    "NHOHCount",
    "NOCount",
]
df = pd.DataFrame(descrs)[interesting_descriptors]
df["Formular"] = formulars
df["smiles"] = data
df.insert(0, "Formular", df.pop("Formular"))
df.insert(0, "smiles", df.pop("smiles"))

In [ ]:
# data selection
# choose which property to filter and how many samples are wanted
property_to_select = "MolWt"
n_samples = 30

# create equially spaced indicies to use in sorted df
idx = np.round(np.linspace(1, len(df) - 1, n_samples + 1)).astype(int)


df_sorted = df.sort_values(property_to_select)
sampled_values = df_sorted.iloc[idx]
print(len(sampled_values))

In [ ]:
sampled_values

In [ ]:
Chem.Draw.MolsToGridImage(
    [Chem.MolFromSmiles(smiles) for smiles in sampled_values["smiles"].values]
)

In [ ]:
sampled_values.to_csv("example_molecules.csv")

In [ ]:
df["MolWt"].hist()

In [ ]:
sampled_values

In [ ]:
mol_list = [Chem.MolFromSmiles(smiles) for smiles in sampled_values["smiles"].values]

In [ ]:
from rdkit.Chem import AllChem
import pathlib

file_path = pathlib.Path("../data/sdw_files")


for i, mol in enumerate(mol_list):
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    AllChem.MMFFOptimizeMolecule(mol)
    writer = Chem.SDWriter(f"molecule_{i}.sdw")
    writer.write(mol)
    writer.close()